# Pre and post processing

https://github.com/facebookresearch/faiss/wiki/Pre--and-post-processing

在某些情形下，需要对Index做前处理或后处理。

## ID映射

默认情况下，faiss会为每个输入的向量记录一个次序id，在使用中也可以为向量指定任意我们需要的id。  
部分index实现了`add_with_ids`方法，为每个向量指定一个`64-bit`的id。搜索时，返回这个指定的id，而不是它的次序id.

In [8]:
import sys
import faiss
import numpy as np 

d = 512
n_data = 2000

data = np.random.rand(n_data, d).astype('float32')
ids = np.arange(100000, 100000+n_data)
print(data.shape)
print(ids.shape)

(2000, 512)
(2000,)


In [9]:
k=6
nlist = 10
quantizer = faiss.IndexFlatIP(d)                                 # quantizer is IndexFlatIP
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2) # IndexIVFFlat is index
index.train(data)
index.add_with_ids(data, ids)                                    # add id to data

D, I = index.search(data[:5], k)

print(I)
print(D)

[[100000 100928 101977 101456 100722 100497]
 [100001 100654 101472 100278 101463 100636]
 [100002 100520 101216 101542 100879 101003]
 [100003 100608 100413 101576 101505 100380]
 [100004 100140 101577 100406 100113 101834]]
[[ 0.       74.08646  75.16429  75.887405 76.40492  77.736786]
 [ 0.       72.32209  73.0237   73.25847  73.73253  75.84412 ]
 [ 0.       75.78934  76.82681  77.3495   77.479324 78.62706 ]
 [ 0.       75.210556 75.411766 75.52989  75.62387  75.93817 ]
 [ 0.       74.38551  76.65866  78.1769   78.58206  79.319275]]


但有些Index不支持`add_with_ids`，比如`IndexFlatL2`，需要与其他Index类型结合，将默认id映射到指定id，用`IndexIDMap`类实现。 
```python
index2 = faiss.IndexFlatL2(data.shape[1]) 
index2.add_with_ids(data, ids)  #报错
```

In [10]:
k=5
index = faiss.IndexFlatL2(d)
index2 = faiss.IndexIDMap(index)  
index2.add_with_ids(data, ids)  
index2.train(data)

D, I = index2.search(data[:5], k)

print(D)
print(I)

[[ 0.       74.08646  74.229675 74.70283  75.16429 ]
 [ 0.       70.893295 72.17247  72.32209  73.0237  ]
 [ 0.       72.326584 72.50299  73.38095  73.95686 ]
 [ 0.       72.17033  72.52927  72.87219  73.795456]
 [ 0.       71.19104  72.67583  74.147606 74.36489 ]]
[[100000 100928 100019 101003 101977]
 [100001 101213 101450 100654 101472]
 [100002 101294 101921 100531 101089]
 [100003 100894 100216 101438 101367]
 [100004 101518 101930 100434 101463]]


## Pre-transforming the data

通常，在索引前，进行数据转换是很有用的。`Transformation`类继承自`VectorTransform`. `VectorTransform`在大小为`d_in`的输入向量集合应用一个转换，输出大小为`d_out`.

Transformation|	Class name|	Comments
---|---|---
random rotation|	RandomRotationMatrix|	useful to re-balance components of a vector before indexing in an IndexPQ or IndexLSH
remapping of dimensions|	RemapDimensionsTransform|	to reduce or increase the size of a vector because the index has a preferred dimension, or to apply a random permutation on dimensions.
PCA	|PCAMatrix|	降维
OPQ rotation|	OPQMatrix|	OPQ applies a rotation to the input vectors to make them more amenable to PQ coding. See [Optimized product quantization](http://www.cv-foundation.org/openaccess/content_cvpr_2013/html/Ge_Optimized_Product_Quantization_2013_CVPR_paper.html), Ge et al., CVPR'13 for more details.

Transformations can be trained from a set of vectors if it makes sense, using the method train. They can be applied to a set of vectors with apply.

An index can be wrapped in a IndexPreTransform index so that the mapping occurs transparently, and training is integrated with the Index training.

### 举例：用PCA降维（带 IndexPreTransform）

例如，输入向量是`2048`维，必须减小到`16`字节，那么，你可以采用PCA降维。

In [11]:
data = np.random.rand(n_data, 2048).astype('float32')
# the IndexIVFPQ will be in 256D not 2048
coarse_quantizer = faiss.IndexFlatL2 (256) 
sub_index = faiss.IndexIVFPQ (coarse_quantizer, 256, 16, 16, 8)
# PCA 2048->256
# 降维后随机旋转 (第四个参数)
pca_matrix = faiss.PCAMatrix (2048, 256, 0, True) 

#- the wrapping index
index = faiss.IndexPreTransform (pca_matrix, sub_index)

# will also train the PCA
index.train(data)  #数据需要是2048维
# PCA will be applied prior to addition
index.add(data)

### 举例：升维

有时, 通过向向量集合插入0来增加维度。This can be useful for:
1. `d`是`4`的倍数，有利于距离计算
2. `d`是`M`的倍数。其中，`M`是PQ的大小

In [12]:
d = 512
M = 8   #M是在维度方向上分割的子空间个数
d2 = int((d + M - 1) / M) * M
remapper = faiss.RemapDimensionsTransform (d, d2, True)
index_pq = faiss.IndexPQ(d2, M, 8)
index = faiss.IndexPreTransform (remapper, index_pq) #后续可以添加数据/索引

## IndexRefineFlat: 搜索结果重排序

当查询一个向量时，it may be useful to re-rank the search results with real distance computations.

下面例子，用一个`IndexPQ`搜索，then reranks the first results by computing real distances:

In [30]:
d = 64                           
nb = 100000                       
nq = 10000                       
np.random.seed(1234)                              # 对生成的数据，每次都一样，可预见，若无此句，每次都不一样
xb = np.random.random((nb, d)).astype('float32')  # 向量集合，矩阵nb*d
xb[:, 0] += np.arange(nb) / 1000                  # 修正向量集合第一列，每项加 0...nb/1000 小数

nbits_per_index = 4
q = faiss.IndexPQ(d, M, nbits_per_index)
rq = faiss.IndexRefineFlat(q)
rq.train(xb)
rq.add(xb)
rq.k_factor = 4

D, I = rq.search(xb[:5], 10)

print(I)
print(D)

[[   0  584 1098  302 1732 2579 2351 2412 2013 2188]
 [   1  277 1063   88    5 1024 1228   46  704  652]
 [   2  134  609  923  276  523  446  334 1322  849]
 [   3  484  527 1246 1145  162  844  699 1197 1601]
 [   4  159  574  918  358 1265 1224 1147 2039 2444]]
[[ 0.         7.811343   8.803015   8.828468  11.423376  14.193947
  14.427416  14.859474  14.988792  16.803171 ]
 [ 0.         6.684581   6.927656   7.647169   8.54887    9.056459
   9.129137   9.189653   9.243975   9.614325 ]
 [ 0.         7.7231803  9.460411   9.658077  10.098377  10.101095
  10.421858  10.527204  11.157595  11.171846 ]
 [ 0.         7.7859573  7.8467503  8.788196   9.411254   9.450055
  10.323664  10.388208  11.065765  13.272972 ]
 [ 0.         7.760465   8.555933   8.569357   8.782854  10.703641
  10.946013  11.725261  12.343815  13.766685 ]]


搜索阶段会先从`IndexPQ`选取4*10个最近邻，然后对每个结果计算真实距离，从中选取10个最好的结果。

注意，`IndexRefineFlat`会保存全部向量集合，内存开销很大。

In [32]:
d = 64                           
nb = 100000                       
nq = 10000                       
np.random.seed(1234)                              # 对生成的数据，每次都一样，可预见，若无此句，每次都不一样
xb = np.random.random((nb, d)).astype('float32')  # 向量集合，矩阵nb*d
xb[:, 0] += np.arange(nb) / 1000                  # 修正向量集合第一列，每项加 0...nb/1000 小数

rq = faiss.IndexFlatL2(d)
rq.add(xb)
D, I = rq.search(xb[:5], 10)
print(I)
print(D)

[[   0  393  363   78  924  364  100  677  491  247]
 [   1  555  277  364  617  175 1063  756   77  191]
 [   2  304  101   13  801  134  365  225  837  397]
 [   3  173   18  182  484   64  527  887  409  316]
 [   4  288  370  531  178  381  175  270   18  364]]
[[0.        7.1751733 7.207629  7.2511625 7.321895  7.351989  7.404186
  7.444352  7.5941343 7.6510983]
 [0.        6.3235645 6.684581  6.7999454 6.8844795 6.919898  6.927656
  7.493     7.4973536 7.5480022]
 [0.        5.7964087 6.391736  7.2815123 7.640502  7.7231803 7.80859
  7.843681  7.8551083 7.8566103]
 [0.        7.2779055 7.5279865 7.6628466 7.7859573 7.790914  7.8467503
  7.968584  7.9748025 8.072303 ]
 [0.        6.7638035 7.2951202 7.3688145 7.3900466 7.46482   7.518219
  7.532466  7.5328097 7.5554886]]


## IndexShards: combining results from several indexes

当数据集分布在多个index中时，需要在每个index中都执行搜索，然后使用`IndexShards`合并结果。同样适用于索引分布在GPUs中，查询可以并行, see index_cpu_to_gpus with shards set to true in GpuClonerOptions.